In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from joblib import Parallel, delayed 
import joblib 

In [4]:
occu = pd.read_excel(f'scheduller/schedule.xlsx',sheet_name="2023")

In [5]:
occu.DAY.unique()

array(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday'], dtype=object)

In [6]:
# Replace multiple values at once
occu.replace({'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}, inplace=True)
# Replace multiple values at once
occu.replace({True: 1, False: 0}, inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_17524\1787995301.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  occu.replace({'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}, inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_17524\1787995301.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  occu.replace({True: 1, False: 0}, inplace=True)


In [7]:
predictor_var= occu.drop(["DATE", "OCCUPANCY"], axis=1) #all columns except the last one
target_var= occu["OCCUPANCY"] #only the last column
X = occu.drop(["DATE", "OCCUPANCY"], axis=1)
y = occu["OCCUPANCY"]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

In [9]:
param_grid = [{"max_depth":[3,4,5, None], "max_features":[2,3]}]
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=123),param_grid = param_grid,cv=10)

In [10]:
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=123),
             param_grid=[{'max_depth': [3, 4, 5, None],
                          'max_features': [2, 3]}])

In [11]:
gs.cv_results_['params']

[{'max_depth': 3, 'max_features': 2},
 {'max_depth': 3, 'max_features': 3},
 {'max_depth': 4, 'max_features': 2},
 {'max_depth': 4, 'max_features': 3},
 {'max_depth': 5, 'max_features': 2},
 {'max_depth': 5, 'max_features': 3},
 {'max_depth': None, 'max_features': 2},
 {'max_depth': None, 'max_features': 3}]

In [12]:
gs.cv_results_['rank_test_score']

array([7, 6, 8, 4, 5, 3, 2, 1], dtype=int32)

In [13]:
gs.best_estimator_

DecisionTreeRegressor(max_features=3, random_state=123)

In [14]:
tree = gs.best_estimator_

In [15]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_features=3, random_state=123)

In [17]:
predictions = tree.predict(X_test)

In [18]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':predictions})
df.head(5) #Check the top 5 predictions and actual values.

,Actual,Predicted
9,100.0,100.0
346,80.0,80.0
266,10.0,10.0
358,8.0,8.5
314,59.5,59.5


In [19]:
# Save the model as a pickle in a file 
joblib.dump(tree, 'model.pkl') 
  
# Load the model from the file 
dtree_from_joblib = joblib.load('model.pkl') 
  
# Use the loaded model to make predictions 
predictions3=dtree_from_joblib.predict(X_test) 

In [20]:
df=pd.DataFrame({'Actual':y_test, 'Predicted3':predictions3})
df.head(5) #Check the top 5 predictions and actual values.

,Actual,Predicted3
9,100.0,100.0
346,80.0,80.0
266,10.0,10.0
358,8.0,8.5
314,59.5,59.5


In [21]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,predictions)))
print('r2_score:', metrics.r2_score(y_test,predictions))

Mean Absolute Error: 0.004545454545454545
Mean Squared Error: 0.0022727272727272726
Root Mean Squared Error: 0.04767312946227961
r2_score: 0.9999986622363828


In [22]:
tree.feature_importances_
pd.Series(tree.feature_importances_,index=predictor_var.columns).sort_values(ascending=False)

DAY               0.583918
PUBLIC HOLIDAY    0.398190
MONTH             0.017892
dtype: float64